<a href="https://colab.research.google.com/github/AdamWilsonLab/GEE-Image-Fusion/blob/main/ee_Downscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example MODIS - LANDSAT data fusion

## Connect to Earth Engine

In [ ]:
import ee
import time

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=jsDGZSGkoJonuvqEewafCGv1o-N3nN3nZ06qNHWT3-I&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi3yH3BuWZzXm1WZulmUBnBtDgQalw4Yuy43o1cbQDN8cLhsw2Rbzc

Successfully saved authorization token.


# Import GEE-Image-Fusion

Import from forked copy in github. 

In [ ]:
import os
from subprocess import getoutput
getoutput("git clone -l -s https://github.com/AdamWilsonLab/GEE-Image-Fusion cloned-repo")
os.chdir('cloned-repo')
from GEE_ImageFusion import *

Common bands between sensors that will be used for fusion would need to add functions for other indices (evi etc.). NDVI is exported as 16 bit int so would have to make sure not to rescale   reflectance bands if that is what you are planning to predict (line 206)  some restructuring of this code would be necessary if the goal was to predict a multiband image but the core functions should work for any number of bands

In [ ]:
region = ee.Geometry.Point([18.419492,-34.266544])
region_rectangle = ee.Geometry.Rectangle([18.293149, -34.386760, 18.778869236816362, -33.8686963851447])
#region = region_rectangle

# define training data temporal bounds broadly
startDate = '2017-03-01'
endDate = '2018-01-01'


commonBandNames = ee.List(['ndvi'])


In [ ]:
# image collections to use in fusion
# NOTE: if using older Landsat and not using NDVI one would have to modify the
# get_paired_collections script because this script harmonizes NDVI from
# L5 & L7 to L8 based on Roy et al. 2016 (see etmToOli and getPaired functions)
landsatCollection = 'LANDSAT/LC08/C01/T1_SR' #'LANDSAT/LC08/C02/T1_L2'
modisCollection = 'MODIS/006/MCD43A4'

# landsat band names including qc band for masking
bandNamesLandsat = ee.List(['blue', 'green', 'red',
                            'nir', 'swir1', 'swir2', 'pixel_qa'])
landsatBands = ee.List([1, 2, 3, 4, 5, 6, 10])

# modis band names
bandNamesModis = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])
modisBands = ee.List([2, 3, 0, 1, 5, 6])

# radius of moving window
# Note: Generally, larger windows are better but as the window size increases,
# so does the memory requirement and we quickly will surpass the memory
# capacity of a single node (in testing 13 was max size for single band, and
# 10 was max size for up to 6 bands)
kernelRadius = ee.Number(10)
kernel = ee.Kernel.square(kernelRadius)
numPixels = kernelRadius.add(kernelRadius.add(1)).pow(2)

# number of land cover classes in scene
coverClasses = 7

# to export the images to an asset we need the path to the assets folder
path = 'users/adammichaelwilson/fusion/'
scene_name = 'NDVI_CapePoint_'


# Get filtered collections

In [ ]:


# sorted, filtered, paired image retrieval
paired = getPaired(startDate, endDate,
                   landsatCollection, landsatBands, bandNamesLandsat,
                   modisCollection, modisBands, bandNamesModis,
                   commonBandNames, region)

subs = makeSubcollections(paired)
#subs_meta = subs.getInfo()
#subs.getInfo()

In [ ]:
subs.length().getInfo()

1

In [ ]:
paired[0].getInfo()

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:32634',
     'crs_transform': [30, 0, 158685, 0, -30, -3717285],
     'data_type': {'max': 1,
      'min': -1,
      'precision': 'float',
      'type': 'PixelType'},
     'dimensions': [7651, 7701],
     'id': 'ndvi'}],
   'id': 'LANDSAT/LC08/C01/T1_SR/LC08_175084_20170417',
   'properties': {'CLOUD_COVER': 3.84,
    'CLOUD_COVER_LAND': 9.52,
    'CloudSnowMaskedPercent': 8,
    'DOY': '107',
    'EARTH_SUN_DISTANCE': 1.003866,
    'ESPA_VERSION': '2_23_0_1a',
    'GEOMETRIC_RMSE_MODEL': 6.751,
    'GEOMETRIC_RMSE_MODEL_X': 5.04,
    'GEOMETRIC_RMSE_MODEL_Y': 4.492,
    'IMAGE_QUALITY_OLI': 9,
    'IMAGE_QUALITY_TIRS': 9,
    'LANDSAT_ID': 'LC08_L1TP_175084_20170417_20170501_01_T1',
    'LEVEL1_PRODUCTION_DATE': 1493670417000,
    'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0',
    'SATELLITE': 'LANDSAT_8',
    'SENSING_TIME': '2017-04-17T08:35:03.4136230Z',
    'SOLAR_AZIMUTH_ANGLE': 40.418468,
    'SOLAR_ZENITH_ANGLE': 54.743801,
 

# Predict and Export Images

In [ ]:

# loop through each list of paired images
num_lists = subs.length().getInfo()
for i in range(0, num_lists):
    # determine the number of modis images between pairs
    num_imgs = ee.List(ee.List(subs.get(i)).get(2)).length()

    # determine the remainder of images, if not groups of 10
    remaining = num_imgs.mod(10)

    # create sequence of starting indices for modis images
    index_seq = ee.List.sequence(0, num_imgs.subtract(remaining), 10)

    # images to be grouped and predicted
    subList = ee.List(ee.List(subs.get(i)).get(2))

    # loop through indices predicting in batches of 10
    for x in range(0, index_seq.length().getInfo()):
        # starting index
        start = ee.Number(index_seq.get(x))

        # ending index
        end = ee.Algorithms.If(start.add(10).gt(num_imgs),
                               num_imgs,
                               start.add(10))

        # group of images to predict
        pred_group = subList.slice(start, end)
        landsat_t01 = ee.List(ee.List(subs.get(i)).get(0))
        modis_t01 = ee.List(ee.List(subs.get(i)).get(1))
        modis_tp = pred_group

        # get the start and end day values and year for the group to use
        # to label the file when exported to asset
        startDay = ee.Number.parse(ee.ImageCollection(pred_group)
                                   .first()
                                   .get('DOY'))
        endDay = ee.Number.parse(ee.ImageCollection(pred_group)
                                 .sort('system:time_start', False)
                                 .first()
                                 .get('DOY'))
        year = ee.Date(ee.ImageCollection(pred_group)
                       .sort('system:time_start', False)
                       .first()
                       .get('system:time_start')).format('Y')

        # start and end day of year
        doys = landsat_t01 \
            .map(lambda img: ee.String(ee.Image(img).get('DOY')).cat('_'))

        # register images
        landsat_t01, modis_t01, modis_tp = registerImages(landsat_t01,
                                                          modis_t01,
                                                          modis_tp)

        # prep landsat imagery (mask and format)
        maskedLandsat, pixPositions, pixBN = prepLandsat(landsat_t01,
                                                         kernel,
                                                         numPixels,
                                                         commonBandNames,
                                                         doys,
                                                         coverClasses)

        # prep modis imagery (mask and format)
        modSorted_t01, modSorted_tp = prepMODIS(modis_t01, modis_tp, kernel,
                                                numPixels, commonBandNames,
                                                pixBN)

        # calculate spectral distance
        specDist = calcSpecDist(maskedLandsat, modSorted_t01,
                                numPixels, pixPositions)

        # calculate spatial distance
        spatDist = calcSpatDist(pixPositions)

        # calculate weights from the spatial and spectral distances
        weights = calcWeight(spatDist, specDist)

        # calculate the conversion coefficients
        coeffs = calcConversionCoeff(maskedLandsat, modSorted_t01,
                                     doys, numPixels, commonBandNames)

        # predict all modis images in modis tp collection
        prediction = modSorted_tp \
            .map(lambda image:
                 predictLandsat(landsat_t01, modSorted_t01,
                                doys, ee.List(image),
                                weights, coeffs,
                                commonBandNames, numPixels))

        # create a list of new band names to apply to the multiband ndvi image
        # NOTE: cant export with names starting with 0
        preds = ee.ImageCollection(prediction).toBands()
        dates = modis_tp.map(lambda img:
                             ee.Image(img).get('system:time_start'))
        predNames = ee.List.sequence(0, prediction.length().subtract(1)) \
            .map(lambda i:
                 commonBandNames\
                     .map(lambda name:
                          ee.String(name)
                          .cat(ee.String(ee.Number(dates.get(i)).format()))))\
            .flatten()


        # status update
        print(ee.String(path).cat(ee.String(scene_name)).cat(year).cat('_').cat(startDay.format()).cat('_').cat(endDay.format()).getInfo())
        
        # export all predictions as a single multiband image
        # each band name corresponds to the timestamp for the image
        task = ee.batch.Export.image.toAsset(
            image=preds.rename(predNames).multiply(10000).toInt16(),
            description=ee.String(scene_name)
                        .cat(year)
                        .cat('_')
                        .cat(startDay.format())
                        .cat('_').cat(endDay.format()).getInfo(),
            assetId=ee.String(path)
                    .cat(ee.String(scene_name))
                    .cat(year)
                    .cat('_')
                    .cat(startDay.format())
                    .cat('_')
                    .cat(endDay.format()).getInfo(),
            region=ee.Image(prediction.get(0)).geometry(),
            scale=30)

        task.start()

        while task.active():
          print('Polling for task (id: {}).'.format(task.id))
          time.sleep(30)
        

users/adammichaelwilson/fusion/NDVI_CapePoint_2017_108_117
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling for task (id: TY6PBYRVJVGNB6UNS5TZOZOL).
Polling fo

In [ ]:
preds.select(['0_ndvi']).getInfo()

In [ ]:
preds.mask(region_rectangle)

In [ ]:
# The folium library can be used to display ee.Image objects on an interactive Leaflet map.
# https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb#scrollTo=VIiyf6azf4mU
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 1,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[18.419492,-34.266544], zoom_start=5)

# Add the elevation model to the map object.
my_map.add_ee_layer(preds.select(['0_ndvi']), vis_params, 'NDVI1')
my_map.add_ee_layer(preds.select(['1_ndvi']), vis_params, 'NDVI2')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)